In [4]:
import os
import dotenv

# Load the environment variables from .venv file
dotenv.load_dotenv('.env')

#GEMINI_API_KEY = 'ok'

# Get the value of CHATGPT_API_KEY from the environment
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

#print(GEMINI_API_KEY)

In [5]:
import glob
import os

ESBM = {}

# Define the path pattern to find .txt files in the data folder
path_pattern = 'data/lmdb/*.txt'

# Use glob to get a list of all .txt files in the data folder
file_list = glob.glob(path_pattern)

# Loop through each file found
for file_path in file_list:
    print(f'Processing file: {file_path}')
    
    # Generate an output filename based on the input filename
    base_name = os.path.basename(file_path)
    output_file_name = f'output_{base_name}'
    
    clean_base_name = base_name.split('.txt')[0]
    print(clean_base_name)
    if clean_base_name not in ESBM:
        ESBM[clean_base_name] = {}

    with open(file_path, 'r') as file:

        for line in file:
            # Split the line on tabs to isolate the URL
            parts = line.split('\t')

            if len(parts) > 1:  # Check if there are at least two parts after splitting
                url = parts[1].strip()  # Get the URL part and remove any extra spaces
                
                # Extract the last part of the URL after the last '/'
                last_part = url.split('/')[-1]
                
                # Remove trailing comma if it exists
                if last_part.endswith(','):
                    last_part = last_part[:-1]
                
                # Replace underscores with spaces if necessary
                # Check if there are underscores and then replace them with spaces
                if '_' in last_part:
                    last_part = last_part.replace('_', ' ')

                # Check if the base_name is already in the dictionary
                
                
                ESBM[clean_base_name][parts[0]] = last_part
                #ids.append(parts[0])
                #data.append(last_part)

        #ESBM[base_name] = data

        #data = []

print(ESBM)


Processing file: data/lmdb/ESBM_benchmark_v1.0.txt
ESBM_benchmark_v1.0
Processing file: data/lmdb/ESBM_benchmark_v1.2.txt
ESBM_benchmark_v1.2
{'ESBM_benchmark_v1.0': {'101': '12398', '102': '12710', '103': '14299', '104': '14438', '105': '2108', '106': '2381', '107': '2572', '108': '4053', '109': '41006', '110': '41408', '111': '41573', '112': '4501', '113': '5232', '114': '6191', '115': '6213', '116': '6468', '117': '70866', '118': '7348', '119': '7963', '120': '8446', '121': '10100', '122': '10846', '123': '29369', '124': '30569', '125': '30598', '126': '31707', '127': '32500', '128': '32648', '129': '33137', '130': '34577', '131': '36141', '132': '38225', '133': '38875', '134': '39725', '135': '416', '136': '43265', '137': '52869', '138': '62725', '139': '67878', '140': '8669'}, 'ESBM_benchmark_v1.2': {'101': '12398', '102': '12710', '103': '14299', '104': '14438', '105': '2108', '106': '2381', '107': '2572', '108': '4053', '109': '41006', '110': '41408', '111': '41573', '112': '450

In [6]:
import math
import numpy as np

import glob
import os
import json  # Import JSON to write output files

import google.generativeai as genai

# Set your API key
genai.configure(api_key=GEMINI_API_KEY)

# Assuming ESBM is a predefined dictionary
for key, value in ESBM.items():
    path_pattern = f'data/lmdb/{key}'
    folder_list = glob.glob(path_pattern)
    
    json_output_list = []  # List to hold output dictionaries

    # Loop through each file found
    for file_path in folder_list:
        print(f'Processing folder: {file_path}')
        
        # Generate an output filename based on the folder name
        base_name = os.path.basename(file_path)

        for key2, value2 in value.items():
            file_list = glob.glob(f'{file_path}/{key2}/*top*.nt')

            for nt_file in file_list:
                print(f'Processing file: {nt_file}')
                with open(nt_file, 'r') as file:
                    rdf_content = file.read()
                
                try:
                    # Define the query
                    query = f"I will give you text - triples and you will respond strictly with coherent text without using your knowledge but only the triples. Triples are: {rdf_content}"

                    # Make the request
                    model = genai.GenerativeModel("gemini-1.5-flash")
                    response = model.generate_content(query)
                    #print(response.text)

                    # Create dictionary for JSON output
                    output_dict = {
                        "id": str(key2),
                        "filename": nt_file,
                        "Title": value[key2],  # Assuming the folder name as title
                        "Answer": response.text,
                        "sparql_query": rdf_content  # Assuming this field is empty for now
                    }
                    json_output_list.append(output_dict)
                    

                except Exception as e:
                    print(f"Error processing file {nt_file}: {e}")
                    break

    # Write the output list to a JSON file for each folder
    json_output_path = os.path.join(file_path, f'{base_name}_gemini_output_gold.json')
    with open(json_output_path, 'w') as json_file:
        json.dump(json_output_list, json_file, indent=4)

    print(f'JSON output written to: {json_output_path}')


Processing folder: data/lmdb/ESBM_benchmark_v1.0
Processing file: data/lmdb/ESBM_benchmark_v1.0/101/101_gold_top10_2.nt
Processing file: data/lmdb/ESBM_benchmark_v1.0/101/101_gold_top5_0.nt
Processing file: data/lmdb/ESBM_benchmark_v1.0/101/101_gold_top5_4.nt
Processing file: data/lmdb/ESBM_benchmark_v1.0/101/101_gold_top5_5.nt
Processing file: data/lmdb/ESBM_benchmark_v1.0/101/101_gold_top10_3.nt
Processing file: data/lmdb/ESBM_benchmark_v1.0/101/101_gold_top5_1.nt
Processing file: data/lmdb/ESBM_benchmark_v1.0/101/101_gold_top10_4.nt
Processing file: data/lmdb/ESBM_benchmark_v1.0/101/101_gold_top10_0.nt
Processing file: data/lmdb/ESBM_benchmark_v1.0/101/101_gold_top5_2.nt
Processing file: data/lmdb/ESBM_benchmark_v1.0/101/101_gold_top10_1.nt
Processing file: data/lmdb/ESBM_benchmark_v1.0/101/101_gold_top5_3.nt
Processing file: data/lmdb/ESBM_benchmark_v1.0/101/101_gold_top10_5.nt
Processing file: data/lmdb/ESBM_benchmark_v1.0/102/102_gold_top5_0.nt
Processing file: data/lmdb/ESBM_ben